# **Apache Spark**
- Autores: Anny Álvarez Nogales, Paula Arias Fernández, Jorge del Castillo Gómez

In [1]:
import numpy as np
from pyspark import SparkContext
sc = SparkContext("local[*]", "Practica1")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/26 10:58:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def readFile(filename):
    text_file = sc.textFile(filename)
    text_file = text_file.map(lambda linea: np.array(linea.split(',')).astype(float))
    text_file= text_file.map(lambda x: (x[0:-1], int(x[-1])))
    return text_file    

CSV separado por comas ","

num filas= 1.000.000, num columnas= 12

Cada celda es un numero. 

La ultima celda de cada fila es la etiqueta (0/1)

FUNCIONES textFile, count, map, flatMap, reduce and reduceByKey. 

# **Ejercicio 1**

In [18]:
text_file= readFile('botnet_tot_syn_l.csv')
text_file

PythonRDD[202] at RDD at PythonRDD.scala:53

In [19]:
def normalize (RDD_Xy):
    '''Normalize rdd'''
    rdd_rows = RDD_Xy.map(lambda row: row[0])  
    sumas = rdd_rows.reduce(lambda x, y: x + y)  
    means_array = sumas / RDD_Xy.count()
    
    rdd_diff = rdd_rows.map(lambda x_array: (x_array - means_array)**2)
    array_diff_sumatorio = rdd_diff.reduce(lambda x_row, y_row: x_row + y_row)
    array_varianza = np.maximum(array_diff_sumatorio / RDD_Xy.count(), 0)

    rdd_final = RDD_Xy.map(lambda row: (
        np.array([(value - means_array[idx_column]) / np.sqrt(array_varianza[idx_column]) 
                  for idx_column, value in enumerate(row[0])]), row[1]
    ) )
    return rdd_final

In [20]:
data = normalize(text_file)
data.take(2)

[(array([-0.79240977, -0.81309371, -0.42245076, -0.46646975, -0.52239296,
         -0.35631957,  0.7370103 ,  0.52834963,  0.82717799,  0.47316616,
          0.15895172]),
  1),
 (array([ 1.4437204 ,  1.116361  ,  0.39127564, -0.46646975, -0.5223921 ,
         -0.35631927,  0.92633194,  1.48012075,  1.02641833, -0.82869545,
          0.15894853]),
  0)]

In [21]:
data_norm_test= data.map(lambda x : x[0])
data_norm_test.stdev()

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [22]:
def compute_sigmoid(z):
    y = (1 /(1 + np.exp(-z)))
    return y  


def predict(w, b, X):
    y = compute_sigmoid(np.dot(X,w) + b)
    if y >= 0.5:
        return 1
    else: 
        return 0


def accuracy(w,b, RDD_Xy):
    y_hat = RDD_Xy.map(lambda x: (predict(w, b, x[0]), x[1]))  
    preds = y_hat.map(lambda x: 1 if x[0] == x[1] else 0)
    correct_preds = preds.reduce(lambda x, y: x + y)    
    total = RDD_Xy.count()
    accuracy = correct_preds / total
    
    return accuracy


def train(RDD_Xy, iterations, learning_rate, lamba_reg):
    # inicializar pesos
    weights= np.random.rand(11)
    biases = np.random.rand(1)   

    for it in range(iterations):
        # número de datos
        m=RDD_Xy.count()
        # calcular predicción (aplicar sigmoide a la multiplicación de los pesos más el sesgo)
        rdd2=RDD_Xy.map(lambda x: (x[0], x[1],compute_sigmoid(np.dot(x[0],weights) + biases)))

        item_2_costFunction = (lamba_reg/(2*11))*np.array([w**2 for w in weights]).sum() 
        
        rdd2 = rdd2.map(lambda x: (x[0], x[1], x[2], -(x[1]*np.log(x[2]) + (1 - x[1])*np.log(1-x[2]))))
        coste=rdd2.map(lambda x:x[3] )
        coste=coste.reduce(lambda x,y:x+y )
        
        # gradientes
        rdd3= rdd2.map(lambda x: ((x[2]-x[1])*x[0],(x[2]-x[1])))
        rdd4= rdd3.reduce(lambda x,y: (x[0]+y[0], x[1]+y[1]))
        weights2 = (weights * lamba_reg)/ len(weights)
        d_weights= ((rdd4[0]/ m) + weights2 ) 
        d_bias= (rdd4[1]) / m
        # actualización de las derivadas
        weights = weights - learning_rate * d_weights
        biases = biases - learning_rate * d_bias


        coste=(coste/m) +  item_2_costFunction
        print(f"Iteration [{it}]: {coste}")
                
    return weights, biases

w, b, = train(data, 10, 1.5, 1)
acc = accuracy(w, b, data)
print ("Accuracy: ", acc)
    

Iteration [0]: [1.29603809]


Iteration [1]: [0.61700224]


Iteration [2]: [0.45214037]


Iteration [3]: [0.40760816]


Iteration [4]: [0.39060638]


...

Iteration [6]: [0.37938249]


Iteration [7]: [0.37756001]


Iteration [8]: [0.37661814]


Iteration [9]: [0.37611815]


Accuracy:  0.92569


# **Ejercicio 2**

In [25]:
import random
def transform(data, m, n_blocks):   
    # calcular n de elementos de cada bloque
    m_per_block = m // n_blocks

    # creamos una lista de índices que representan las filas del conjunto de datos. Tiene valores del 0 al m-1 (m es el número total de filas)
    indices = list(range(m)) 
    # mezclamos los índices para asegurarnos de que los datos se distribuyan de forma aleatoria entre los bloques
    random.shuffle(indices)

    # la operación modulo % asegura que el índice de cada fila se asigna a un bloque específico.
    # el resultado de 'indices.pop() % n_blocks' será un valor entre 0 y (n_blocks-1)
    # el operador hace que los datos se distribuyan entre los bloques de manera UNIFORME
    # los datos se asignen cíclicamente a bloques (0, 1, 2,... hasta n-1), y como esta desordenado ya se aplica el shuffle.

    # ahora se tiene el resultado de la operacion modulo (y se añade como clave a cada fila)
    indexed_data = data.map(lambda x: (indices.pop() % n_blocks, x)) 
    # 'x' es una fila del RDD original, y 'indices.pop() % n_blocks' le asigna un bloque aleatorio 

    return indexed_data.cache()


In [26]:
def get_block_data(data_cv, i, n_blocks):
    #agrupamos los datos por bloque usando el valor de la clave (n de bloque) y almacenamos los datos al lado
    #tengo (bloque/slice , dato (array y etiqueta))
    grouped_data = data_cv.map(lambda x: (x[0], [x[1]])) 

    
    #reduceByKey' para concatenar los elementos de la misma clave (bloque), es decir, agrupamos
    #los datos que tienen el mismo valor de bloque.
    grouped_data = grouped_data.reduceByKey(lambda a, b: a + b) 
    #aqui tenemos clave (n de bloque, lista de datos (filas con su array de 11 elementos, etiqueta))
    
    #en cada iteracion de este bucle trabajo con un slice o bloque 
    #tomamos todos los elementos cuyo bloque no sea el valor 'i' actual.
    tr_data = grouped_data.map(lambda x: (x[0], [item for item in x[1] if x[0] != i]))
    test_data = grouped_data.map(lambda x: (x[0], [item for item in x[1] if x[0] == i])) # el resto de test (elementos con clave el i)

    # hasta ahora tenia lista de listas para cada bloque (al agrupar con reducebykey)
    #aplano y tengo solo los datos no la clave del bloque (ya no se va a usar)
    tr_data = tr_data.flatMap(lambda x: x[1])  
    test_data = test_data.flatMap(lambda x: x[1])  
    
    return tr_data, test_data
    return tr_data, test_data

In [28]:
# read data
path = 'botnet_tot_syn_l.csv'
data = readFile(path)

# standardize
data = normalize(data)

num_blocks_cv = 10
data_cv = transform(data, data.count(), num_blocks_cv)

nIter = 10
learningRate = 1.5
lamba_reg = 0

accuracies = []

for i in range(num_blocks_cv):
    tr_data, test_data = get_block_data(data_cv, i, num_blocks_cv)
    
    print(f"\nSlice {i+1}:")
    
    w,b = train(tr_data, nIter, learningRate, lamba_reg)
    acc = accuracy(w, b, test_data)
    print("acc :", acc)
    accuracies.append(acc)

avg_acc = sum(accuracies) / num_blocks_cv
print("average acc: ", avg_acc)
    

Exception in thread "serve RDD 300" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:474)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)



Slice 1:


Iteration [0]: [1.55114774]


Iteration [1]: [0.82314099]


Iteration [2]: [0.50744305]


...

Iteration [6]: [0.24640513]


Iteration [7]: [0.23495854]


Iteration [8]: [0.22614513]


Iteration [9]: [0.21912989]


acc : 0.9281296546553952
average acc:  0.928656591604109
